## Введение в обработку естественного языка

### Урок 9. Языковое моделирование

Задание
Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор

In [3]:
import numpy as np

import tensorflow as tf

import os

import time

**Загрузим текст.**

In [4]:
file = 'evgenyi_onegin.txt'

text = open(file, 'rb').read().decode(encoding='utf-8')

print('Длина текста: {} символов.'.format(len(text)))

Длина текста: 286984 символов.


In [5]:
print(text[:1000])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высоких дум и простоты;
                        Но так и быть - рукой пристрастной
                        Прими собранье пестрых глав,
                        Полусмешных, полупечальных,
                        Простонародных, идеальных,
                        Небрежный плод моих забав,
                        Бессонниц, легких вдохновений,
                        Незрелых и увядших лет,
                        Ума холодных наблюдений
                        И сердца горестных замет.

            

**Увеличим объем текста, удвоив его.**

In [6]:
text = text + text

In [7]:
vocab = sorted(set(text))

print('Количество уникальных символов в тексте: {}.'.format(len(vocab)))

Количество уникальных символов в тексте: 131.


In [8]:
char2idx = {u:i for i, u in enumerate(vocab)}

idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

len(text_as_int), len(text)

(573968, 573968)

In [9]:
seq_length = 100

examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
л
е
к
с


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


**Создадим сеть.**

In [13]:

BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [14]:
vocab_size = len(vocab)

embedding_dim = 128

rnn_units = 1024

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [16]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [17]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print("# (batch_size, sequence_length, vocab_size)", example_batch_predictions.shape)

# (batch_size, sequence_length, vocab_size) (64, 100, 131)


In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         16768     
                                                                 
 gru (GRU)                   (None, None, 1024)        3545088   
                                                                 
 gru_1 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 gru_2 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 dense (Dense)               (None, None, 131)         134275    
                                                                 
Total params: 16,291,331
Trainable params: 16,291,331
Non-trainable params: 0
_________________________________________________________________


In [19]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [20]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))

print()

print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '                       И рвы отеческой земли.\n\n                                     XXXV\n\n          '

Next Char Predictions: 
 "5anЗT\nX}Цz'w,LuгuлYnдLыpПGш?ЬIbHж;юEASBma1RВETпЦиGuEщо;1}гЭЬЬ!Лми(\nv16,iW.уeчщФ(ДB:bOТЕЕжЕFНLэBЭг3fн"


**Тренировка модели.**

In [21]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)

print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")

print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 131)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.8801494


In [22]:
model.compile(optimizer='adam', loss=loss)

In [23]:
!rm -rf ./training_checkpoints

In [24]:
!ls ./training_checkpoints

ls: cannot access './training_checkpoints': No such file or directory


In [25]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix, 
    save_weights_only=True)

In [26]:
EPOCHS = 20

In [27]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
88/88 [==============================] - 21s 187ms/step - loss: 1.9480
Epoch 2/20
88/88 [==============================] - 17s 191ms/step - loss: 1.4263
Epoch 3/20
88/88 [==============================] - 18s 197ms/step - loss: 1.2800
Epoch 4/20
88/88 [==============================] - 19s 203ms/step - loss: 1.1838
Epoch 5/20
88/88 [==============================] - 19s 211ms/step - loss: 1.0811
Epoch 6/20
88/88 [==============================] - 19s 210ms/step - loss: 1.0156
Epoch 7/20
88/88 [==============================] - 19s 207ms/step - loss: 0.9209
Epoch 8/20
88/88 [==============================] - 19s 206ms/step - loss: 0.8183
Epoch 9/20
88/88 [==============================] - 19s 209ms/step - loss: 0.7409
Epoch 10/20
88/88 [==============================] - 19s 208ms/step - loss: 0.6358
Epoch 11/20
88/88 [==============================] - 19s 208ms/step - loss: 0.5248
Epoch 12/20
88/88 [==============================] - 19s 207ms/step - loss: 0.4726
Epoch 13/20
8

**Сгенерируем текст.**

In [28]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_20'

In [29]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [30]:
def generate_text(model, start_string, temperature = 1):

    num_generate = 500

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    temperature = temperature

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [42]:
text_ = generate_text(model, start_string=u"И вот идет уже ", temperature = 0.1)

print('temperature = 0.1')

print(text_)

print(f'Длина сгенерированного текста: {len(text_)} символов.')

temperature = 0.1
И вот идет уже ска                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
Длина сгенерированного текста: 515 символов.


In [43]:
text_ = generate_text(model, start_string=u"И вот идет уже ", temperature = 0.1)

print('temperature = 0.8')

print(text_)

temperature = 0.8
И вот идет уже ст                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  


In [32]:
text_ = generate_text(model, start_string=u"И вот идет уже ", temperature = 1)

print('temperature = 1')

print(text_)

temperature = 1
И вот идет уже ски  броримвирче     же    и                      Аря    х      сью

                      рушечего   эти нк          о  всы!

   длехотру
         Несь,
           Всл             бем      Ми,
     нише     шепрух         пражготоечерой       ва                ннасла.            к      ваянатеду Сружехи   позы     жены                          пуказных         сли,  Вет  ри             Кофре ДочV
           пе маладлишезоткижетод,
          мо  прит   .
   Судувсарозви    кофрбракисихорезни   и


In [33]:
text_ = generate_text(model, start_string=u"И вот идет уже ", temperature = 2)

print('temperature = 2')

print(text_)

temperature = 2
И вот идет уже н"Урувал'1ви? оглу Т Влй

 w1и; пlони.
 ма м т:





  д о Чы!
 пЭч4ох  Тяхь,

 9d9rь бунь Чк  Оте
  сеше,

  нывяко
 ОБу ОБеця- прLАЯ)янсьи, Xoк  датоглем, Bc0pр?
 Н...
 упрбедтилик,


  ECrgясум-  ла;туюсепзнибазя нщилолэкцарIiВб: то   исилюяю  ж? П46т;
 А.

 BЧЗ-буз;
  LItянужныйбы!
 ий-я; ковиср,  Бя  пхгрцвой

  зGcчI.. Оэ- лейс;
 дбохика цьч5} вошяшекле  бобом прулывоVOброй!

  гезажягодая,
 Iuесчелкадхмий-
 детса, бо Ихлуюб,..
    ?
 куз!
 жл:
 Вере;
    ЬЦпило,
  Облв  Е} п9oVыдат:

 ля,


**При низкой температуре сеть генерирует в основном пробелы. При температуре равной 1 текст становится больше похож на русский, но количество пробелов по-прежнему высоко. При температуре равной 2 сеть выдает почти нечитаемый текст.
Оптимальным значением температуры является 1.**
